                                              TESTING MASK-RCNN

***IMPORTING REQUIRED PACKAGES & INITIAZING THE ROOT DIRECTORY AS WELL WEIGHT'S PATH***

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
# import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn.model import mold_image
from numpy import expand_dims
from numpy import mean

#import custom

# Root directory of the project
ROOT_DIR = "C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1"

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")


WEIGHTS_PATH = "C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/logs/object20220119T2303/mask_rcnn_object_0008.h5"   # change it


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import keras
print(keras.__version__)

In [ ]:
import cv2
 
# read image
img = cv2.imread('C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/Dataset/train/frame0.png', cv2.IMREAD_UNCHANGED)
 
# get dimensions of image
dimensions = img.shape
 
# height, width, number of channels in image
height = img.shape[0]
width = img.shape[1]
channels = img.shape[2]
 
print('Image Dimension    : ',dimensions)
print('Image Height       : ',height)
print('Image Width        : ',width)
print('Number of Channels : ',channels)

In [ ]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    IMAGES_PER_GPU = 1

    NUM_CLASSES = 1 + 2  # Background + spacecraft/satellite and non_satellite_body

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
config = CustomConfig()
CUSTOM_DIR = os.path.join(ROOT_DIR, "/Dataset/")
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.7

config = InferenceConfig()
config.display()

In [ ]:
# Code for Customdataset class. Same code is present in custom.py file also
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "satellite")
        self.add_class("object", 2, "non_satellite_body")
        #self.add_class("object", 3, "phone")

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        annotations1 = json.load(open('C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/Dataset/val/via_region_data.json'))

        annotations = list(annotations1.values())  # don't need the dict keys

        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
           
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"satellite": 1,"non_satellite_body": 2}#,"phone": 3}
            num_ids = [name_dict[a] for a in objects]

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object", 
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
       
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):

        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [ ]:
# Inspect the model in training or inference modes values: 'inference' or 'training'
TEST_MODE = "inference"
ROOT_DIR = "C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/Dataset"

def get_ax(rows=1, cols=1, size=16):
  """Return a Matplotlib Axes array to be used in all visualizations in the notebook.  Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax

# Load validation dataset
# Must call before using the dataset
CUSTOM_DIR = "C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/Dataset"
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR, "val")
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
config = CustomConfig()
#LOAD MODEL. Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

In [ ]:
# Load COCO weights Or, load the last model you trained
weights_path = WEIGHTS_PATH
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

***READING AND SHOWING AN IMAGE***

In [ ]:
import skimage.io
import matplotlib.pyplot

img = skimage.io.imread("C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/Dataset/val/frame1200.png")

matplotlib.pyplot.imshow(img)
matplotlib.pyplot.show()

***READING A JSON FILE***

In [ ]:
import json
import numpy as np
import pandas as pd

train_set_f = open(r"C:\Users\user\Desktop\EVERYTHING\APC\Mtech\Mtech-Sem3\DISSERTATION\FINAL_DATASET\project\PROJECT_1\Dataset\train\via_region_data.json")

data = json.load(train_set_f)

table_output = pd.DataFrame(data)


In [ ]:
data

In [ ]:
table_output

In [ ]:
#Now, we are ready for testing our model on any image.

#RUN DETECTION
image_id = random.choice(dataset.image_ids)
#image_id = 'D:/MaskRCNN-aar/Dataset/val/1.jfif'
print("image id is :",image_id)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id,dataset.image_reference(image_id)))

# Run object detection
results = model.detect([image], verbose=1)



In [ ]:
# Display results
x = get_ax(1)
r = results[0]
ax = plt.gca()
#visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'], ax=ax, title="Predictions")

visualize.display_instances(image, r['rois'],r['masks'], r['class_ids'], dataset.class_names, r['scores'], ax=ax, title="Predictions")


log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)
# This is for predicting images which are not present in dataset
path_to_new_image = 'C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/Dataset/val_sample/frame1500.png'
image1 = cv2.imread(path_to_new_image)

# Run object detection
print(len([image1]))
results1 = model.detect([image1], verbose=1)

# Display results
ax = get_ax(1)
r1 = results1[0]
visualize.display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'],
dataset.class_names, r1['scores'], ax=ax, title="Predictions1")

In [ ]:
# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		APs.append(AP)
	# calculate the mean AP across all images
# 	mAP = mean(APs)
# 	AP = APs
	return APs

In [ ]:
# evaluate model on validation dataset
val_AP = evaluate_model(dataset, model, config)
TOTAL = 0.0
for i in range(0,len(val_AP)):
    TOTAL = TOTAL + val_AP[i]
print("Validation AP: ", TOTAL)
val_mAP = TOTAL/len(val_AP)
print("Validation mAP: %.3f" % val_mAP)

In [ ]:
print("Each Validation AP: ", val_AP)

In [ ]:
# Code for Customdataset class. Same code is present in custom.py file also
class CustomDataset_train(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "satellite")
        self.add_class("object", 2, "non_satellite_body")
        #self.add_class("object", 3, "phone")

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        annotations1 = json.load(open('C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/Dataset/train/via_region_data.json'))

        annotations = list(annotations1.values())  # don't need the dict keys

        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
           
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"satellite": 1,"non_satellite_body": 2}#,"phone": 3}
            num_ids = [name_dict[a] for a in objects]

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object", 
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
       
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):

        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [ ]:
CUSTOM_DIR = "C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/Dataset"
dataset = CustomDataset_train()
dataset.load_custom(CUSTOM_DIR, "train")
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
config = CustomConfig()
#LOAD MODEL. Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

In [ ]:
# Load COCO weights Or, load the last model you trained
weights_path = WEIGHTS_PATH
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
# evaluate model on TRAIN dataset
train_AP = evaluate_model(dataset, model, config)
TOTAL = 0.0
for i in range(0,len(train_AP)):
    TOTAL = TOTAL + train_AP[i]
print("Train AP: ", TOTAL)
train_mAP = TOTAL/len(train_AP)
print("Train mAP: %.3f" % train_mAP)

In [ ]:
from mrcnn.visualize_cv2_old_new import display_instances

In [ ]:
capture = cv2.VideoCapture('C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/Pendrive_files/2021-10-29_162820_498OMI_1.avi')
frame_count = 0
size = (
    int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
codec = cv2.VideoWriter_fourcc(*'DIVX')
output = cv2.VideoWriter('C:/Users/user/Desktop/EVERYTHING/APC/Mtech/Mtech-Sem3/DISSERTATION/FINAL_DATASET/project/PROJECT_1/Masked/Masked_Full_Video_demo_10.avi', codec, 60.0, size)

while(capture.isOpened()):
    ret, frame = capture.read()
    if ret:
        frame_count+=1
        if frame_count > 0:

        # add mask to frame
            results = model.detect([frame], verbose=0)
            r = results[0]
            frame = display_instances(
                frame, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores']
            )
            output.write(frame)
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            #plt.imshow(cv2.cvtColor(frame, cv2.IMREAD_UNCHANGED))
        
            plt.show()
            #cv2.imshow('frame', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        break

capture.release()
output.release()
cv2.destroyAllWindows()